## Understand Chat Eval

Before I try mid training the d20 model I want to get the chat eval stuff in place. In [speedrun.sh](https://github.com/karpathy/nanochat/blob/master/speedrun.sh) he calls [chat_eval.py](https://github.com/karpathy/nanochat/blob/master/scripts/chat_eval.py) after midtraining. Could I also call it before to see the difference?

I see I'm now going to add the ARC task that I skipped before and also `humaneval.py`

#### MyTokenizer.render_for_completion()

See I will now need this. Go add.

In [1]:
import sys
sys.path.append('../my_nanochat')
from my_nanochat.my_tokenizer import get_tokenizer
from my_tasks.my_spellingbee import MySimpleSpelling

In [2]:
tokenizer = get_tokenizer()
task = MySimpleSpelling()
task[0]

{'messages': [{'role': 'user', 'content': 'Spell the word: baggers'},
  {'role': 'assistant', 'content': 'baggers:b,a,g,g,e,r,s'}]}

In [4]:
# call render_for_completion, we want the final message to be
# gone but do want to end with an assistant start token
tokens = tokenizer.render_for_completion(task[0])
tokenizer.decode(tokens)

'<|bos|><|user_start|>Spell the word: baggers<|user_end|><|assistant_start|>'

#### run_generative_eval()

Good place to start. Will copy this code into `my_chat_eval.py` and then try it out.

In [1]:
import sys
sys.path.append('../my_nanochat')
from my_nanochat.my_tokenizer import get_tokenizer
from my_tasks.my_gsm8k import MyGSM8K
from scripts.my_chat_eval import run_generative_eval
from my_nanochat.my_checkpoint_manager import load_model
from my_nanochat.my_common import compute_init, autodetect_device_type
from my_nanochat.my_engine import Engine

In [2]:
task = MyGSM8K(subset="main", split="test")

In [3]:
task[0]

{'messages': [{'role': 'user',
   'content': "Darrell and Allen's ages are in the ratio of 7:11. If their total age now is 162, calculate Allen's age 10 years from now."},
  {'role': 'assistant',
   'content': [{'type': 'text',
     'text': 'The total ratio representing their ages is 7+11= '},
    {'type': 'python', 'text': '7+11'},
    {'type': 'python_output', 'text': '18'},
    {'type': 'text',
     'text': "18\nSince the fraction of the ratio that represents Allen's age is 11/18, Allen's current age is 11/18*162 = "},
    {'type': 'python', 'text': '11/18*162'},
    {'type': 'python_output', 'text': '99'},
    {'type': 'text',
     'text': '99\nIf Allen is currently 99 years old, in 10 years he will be 99+10 = '},
    {'type': 'python', 'text': '99+10'},
    {'type': 'python_output', 'text': '109'},
    {'type': 'text', 'text': '109 years old\n#### 109'}]}]}

In [4]:
device_type = autodetect_device_type() 
_, _, _, _, device = compute_init(device_type)
model, tokenizer, meta_data = load_model('base', model_tag='d4', device=device, phase='eval')
engine = Engine(model, tokenizer)
run_generative_eval(
    task,
    tokenizer,
    model,
    engine,
    num_samples=1,
    max_new_tokens=128,
    temperature=0,
    top_k=50,
    max_problems=5)

Autodetected device type: mps
loading the model from /Users/ericsilberstein/.cache/my_nanochat/base_checkpoints/d4 with step 500
Building model with config: {'sequence_len': 128, 'vocab_size': 65536, 'n_layer': 4, 'n_head': 2, 'n_kv_head': 2, 'n_embd': 256}
Rank 0 | 0/5 (0.00%)]
final: 0/5 (0.00%)


0.0

^ That stay on the same line thing works in Jupyter. Guess not a surprise because have seen that before.

In [5]:
run_generative_eval(
    task,
    tokenizer,
    model,
    engine,
    num_samples=2,
    max_new_tokens=128,
    temperature=0,
    top_k=50,
    max_problems=5)

Rank 0 | 0/5 (0.00%)]
final: 0/5 (0.00%)


0.0

#### run_categorical_eval()

Copy this code into `my_chat_eval.py` and then try it out.

In [7]:
import torch

In [13]:
# see this tensor slicing technique used to "focus" logits, didn't realize you could do this
# maybe not true, have used this before with torch and numpy I think, just thought of it different
torch.tensor([1,2,3,4,5], dtype=torch.long)[[1,4]]

tensor([2, 5])

In [1]:
import sys
sys.path.append('../my_nanochat')
from my_nanochat.my_tokenizer import get_tokenizer
from my_tasks.my_mmlu import MyMMLU
from scripts.my_chat_eval import run_categorical_eval
from my_nanochat.my_checkpoint_manager import load_model
from my_nanochat.my_common import compute_init, autodetect_device_type
from my_nanochat.my_engine import Engine

In [2]:
task = MyMMLU(subset="all", split="test")

In [3]:
task[0]

{'messages': [{'role': 'user',
   'content': 'Multiple Choice question: Positronium is an atom formed by an electron and a positron (antielectron). It is similar to the hydrogen atom, with the positron replacing the proton. If a positronium atom makes a transition from the state with n=3 to a state with n=1, the energy of the photon emitted in this transition is closest to\n- 6.0 e=A\n- 6.8 eV=B\n- 12.2 eV=C\n- 13.6 eV=D\n\nRespond only with the letter of the correct answer.'},
  {'role': 'assistant', 'content': 'A'}],
 'subject': 'college_physics',
 'letters': ('A', 'B', 'C', 'D')}

In [9]:
device_type = autodetect_device_type() 
_, _, _, _, device = compute_init(device_type)
model, tokenizer, meta_data = load_model('base', model_tag='d4', device=device, phase='eval')
engine = Engine(model, tokenizer)
run_categorical_eval(
    task,
    tokenizer,
    model,
    batch_size=1,
    max_problems=1000)

Autodetected device type: mps
loading the model from /Users/ericsilberstein/.cache/my_nanochat/base_checkpoints/d4 with step 500
Building model with config: {'sequence_len': 128, 'vocab_size': 65536, 'n_layer': 4, 'n_head': 2, 'n_kv_head': 2, 'n_embd': 256}
final: 266/1000 (26.60%)


0.266

### ARC Task

Go implement the ARC task. Skipped before, but did look at the dataset in `challenge-26-understand-midtrain/understand-midtrain.ipynb`

In [11]:
import sys
sys.path.append('../my_nanochat')
from my_tasks.my_arc import MyARC

In [12]:
task = MyARC(subset="ARC-Easy", split="test")

In [13]:
task[0]

{'messages': [{'role': 'user',
   'content': 'Multiple Choice question: A force of 5 N is required to increase the speed of a box from a rate of 1.0 m/s to 3.0 m/s within 5 s along a level surface. What change would most likely require additional force to produce the same results?\n- reduce the mass of the box=A\n- increase the mass of the box=B\n- make the surfaces of the box smooth=C\n- make the surface of the floor smooth=D\n\nRespond only with the letter of the correct answer.'},
  {'role': 'assistant', 'content': 'B'}],
 'letters': ['A', 'B', 'C', 'D']}

In [15]:
task.evaluate(task[0], 'A')

False

In [16]:
task.evaluate(task[0], 'B')

True

### HumanEval Task

See what it is and implement.

His comment: Btw this dataset is a misnomer and has nothing to do with humans. It is a coding benchmark.

In [1]:
import os
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
from datasets import load_dataset
ds = load_dataset('openai/openai_humaneval', split='test')

Generating test split:   0%|          | 0/164 [00:00<?, ? examples/s]

In [2]:
ds[0]

{'task_id': 'HumanEval/0',
 'prompt': 'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n',
 'canonical_solution': '    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n',
 'test': "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True\n    assert 

In [4]:
print(ds[0]['prompt'])

from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """



Going to cheat and just copy and paste all of `humaneval.py` and `execution.py` at least for now.

In [5]:
import sys
sys.path.append('../my_nanochat')
from my_nanochat.execution import execute_code
from my_tasks.humaneval import HumanEval

In [11]:
execute_code('2+3')

ExecutionResult(success=True)

In [12]:
execute_code('print(2+3)')

ExecutionResult(success=True, stdout='5\n')

In [14]:
execute_code('import os; print(os.getcwd())')

ExecutionResult(success=False, error="TypeError: 'NoneType' object is not callable")

In [17]:
execute_code('print(2+3')

ExecutionResult(success=False, error="SyntaxError: '(' was never closed (<string>, line 1)")

In [18]:
task = HumanEval()

In [19]:
task[0]

{'messages': [{'role': 'user',
   'content': '\n\ndef below_threshold(l: list, t: int):\n    """Return True if all numbers in the list l are below threshold t.\n    >>> below_threshold([1, 2, 4, 10], 100)\n    True\n    >>> below_threshold([1, 20, 4, 10], 5)\n    False\n    """\n'},
  {'role': 'assistant',
   'content': '\n\ndef below_threshold(l: list, t: int):\n    """Return True if all numbers in the list l are below threshold t.\n    >>> below_threshold([1, 2, 4, 10], 100)\n    True\n    >>> below_threshold([1, 20, 4, 10], 5)\n    False\n    """\n\n    for e in l:\n        if e >= t:\n            return False\n    return True\n'}],
 'entry_point': 'below_threshold',
 'test': '\n\nMETADATA = {}\n\n\ndef check(candidate):\n    assert candidate([1, 2, 4, 10], 100)\n    assert not candidate([1, 20, 4, 10], 5)\n    assert candidate([1, 20, 4, 10], 21)\n    assert candidate([1, 20, 4, 10], 22)\n    assert candidate([1, 8, 4, 10], 11)\n    assert not candidate([1, 8, 4, 10], 10)\n\n'}

In [20]:
def print_conversation(conversation):
    for message in conversation['messages']:
        print(f"_____{message['role']}_____")
        content = message['content']
        if isinstance(content, str):
            print(content)
        else:
            assert isinstance(content, list)
            for part in content:
                print(f"{part['type']}: {part['text']}")
        print()

In [21]:
print_conversation(task[0])

_____user_____


def below_threshold(l: list, t: int):
    """Return True if all numbers in the list l are below threshold t.
    >>> below_threshold([1, 2, 4, 10], 100)
    True
    >>> below_threshold([1, 20, 4, 10], 5)
    False
    """


_____assistant_____


def below_threshold(l: list, t: int):
    """Return True if all numbers in the list l are below threshold t.
    >>> below_threshold([1, 2, 4, 10], 100)
    True
    >>> below_threshold([1, 20, 4, 10], 5)
    False
    """

    for e in l:
        if e >= t:
            return False
    return True




In [26]:
task.evaluate(task[0], """
def below_threshold(l: list, t: int):
    for e in l:
        if e >= t:
            return False
    return True
""")

True

In [51]:
task.evaluate(task[0], """
def below_threshold(l: list, t: int):
    return not any(e >= t for e in l)
""")

True

### Back to `chat_eval.py`

copy `run_chat_eval()`

In [53]:
import sys
sys.path.append('../my_nanochat')
from scripts.my_chat_eval import run_chat_eval
from my_nanochat.my_checkpoint_manager import load_model
from my_nanochat.my_common import compute_init, autodetect_device_type
from my_nanochat.my_engine import Engine

In [54]:
device_type = autodetect_device_type() 
_, _, _, _, device = compute_init(device_type)
model, tokenizer, meta_data = load_model('base', model_tag='d4', device=device, phase='eval')
engine = Engine(model, tokenizer)

Autodetected device type: mps
loading the model from /Users/ericsilberstein/.cache/my_nanochat/base_checkpoints/d4 with step 500
Building model with config: {'sequence_len': 128, 'vocab_size': 65536, 'n_layer': 4, 'n_head': 2, 'n_kv_head': 2, 'n_embd': 256}


In [55]:
run_chat_eval('MMLU', model, tokenizer, engine, max_problems=5)

final: 2/5 (40.00%)


0.4

In [56]:
run_chat_eval('HumanEval', model, tokenizer, engine, max_problems=5)

Rank 0 | 0/5 (0.00%)]
final: 0/5 (0.00%)


0.0

### main

Add the stuff so it can run as a standalone script.

In [57]:
import os
os.environ["PYTHONPATH"] = "../my_nanochat"

In [60]:
!python -m scripts.my_chat_eval --source=mid --batch-size=1 --model-tag=d4 --max-problems=5

Autodetected device type: mps
loading the model from /Users/ericsilberstein/.cache/my_nanochat/mid_checkpoints/d4 with step 9
Building model with config: {'sequence_len': 128, 'vocab_size': 65536, 'n_layer': 4, 'n_head': 2, 'n_kv_head': 2, 'n_embd': 256}
final: 0/5 (0.00%)
ARC-Easy accuracy: 0.00%
final: 0/5 (0.00%)
ARC-Challenge accuracy: 0.00%
final: 2/5 (40.00%)
MMLU accuracy: 40.00%
Rank 0 | 0/5 (0.00%)]
final: 0/5 (0.00%)
GSM8K accuracy: 0.00%
Rank 0 | 0/5 (0.00%)]
final: 0/5 (0.00%)
HumanEval accuracy: 0.00%
Rank 0 | 0/5 (0.00%)]
final: 0/5 (0.00%)
SpellingBee accuracy: 0.00%


### Make base_eval able to run as standalone script

Unrelated to chat eval, I'm thinking that after midtraining I'll want to see the CORE metrics. I skipped making `my_base_eval.py` able to run as a script earlier. Add now.

In [71]:
import os
os.environ["PYTHONPATH"] = "../my_nanochat"

In [76]:
!python -m scripts.my_base_eval \
    --source=base \
    --model-tag=d4 \
    --max-per-task=20 \
    --tasks-to-run="hellaswag_zeroshot|jeopardy"

Autodetected device type: mps
loading the model from /Users/ericsilberstein/.cache/my_nanochat/base_checkpoints/d4 with step 500
Building model with config: {'sequence_len': 128, 'vocab_size': 65536, 'n_layer': 4, 'n_head': 2, 'n_kv_head': 2, 'n_embd': 256}
Evaluating: hellaswag_zeroshot (0-shot, type: multiple_choice)... accuracy: 0.3500 | centered: 0.1333 | time: 2.03s
Evaluating: jeopardy (10-shot, type: language_modeling)... accuracy: 0.0000 | centered: 0.0000 | time: 1.34s
CORE metric: 0.0667
centered results:
{
    "hellaswag_zeroshot": 0.13333332538604736,
    "jeopardy": 0.0
}


Code added as part of this challenge:

- added `render_for_completion()` to `MyTokenizer`
   
- added `my_arc.py` for ARC task skipped earlier

- copied (breaking rule!) `humaneval.py` for HumanEval tasks and `execution.py` which it uses to execute python programs

- all that needed to create `my_chat_eval.py`

- also added made `my_base_eval.py` able to run as a standalone script